In [1]:
import pandas as pd
import geopandas as gpd
import geohash
import numpy as np
import folium
from shapely.wkt import loads
from shapely import wkt
import shapely.geometry
from shapely.geometry import Polygon
from IPython.display import display
pd.options.display.max_columns = None
import contextily as cx
import math
from matplotlib import pyplot
from scipy import stats
import math
from typing import List, Set
from shapely.geometry import box
from shapely import prepared
import pyproj
import matplotlib.pyplot as plt
from scipy.spatial import cKDTree
from matplotlib.ticker import PercentFormatter

In [11]:
grid_geo_data = gpd.read_file("./cell_coverage.csv")
grid_geo_data["geometry"] = grid_geo_data["geometry"].apply(wkt.loads)

grid_geo_data = gpd.GeoDataFrame(grid_geo_data, geometry="geometry", crs="EPSG:4326")

grid_geo_data.head(2)

,grid_cell,grid,cell_name,cilac,avg_rsrp,avg_rsrq,avg_sinr,event_count,grid_event_count,cell_event_count,avg_rsrp_grid,avg_rsrq_grid,avg_sinr_grid,avg_rsrp_cell,avg_rsrq_cell,avg_sinr_cell,perc_cell_events,perc_grid_events,distance_to_cell,grid_max_distance_to_cell,grid_min_distance_to_cell,cell_max_distance_to_cell,perc_cell_max_dist,cell_angle_to_grid,grid_bearing_diff,cell_count,band_cell_count,latitude,longitude,geometry
0,9wfvbxf_4602116125,9wfvbxf,DNGJT00002B_n71_F_3,4602116125,-115.0,-12.0,1.0,6.0,6.0,7350.0,-115.0,-12.0,1.0,-105.27816412745713,-11.350908054779408,5.174766296195599,8.163265306122449E-4,1.0,4607.806419544988,4607.806419544988,4607.806419544988,10684.002854735672,0.43128090493747695,252.74327998414196,47.256720015858036,1,1,39.02275085449219,-108.60740661621094,"POLYGON ((-108.60809 39.02206, -108.60809 39.0..."
1,9wfwvmz_4602116192,9wfwvmz,DNGJT00005A_n70_AWS-4_UL15_1,4602116192,-111.0,-10.0,14.0,3.0,9.0,669.0,-108.5,-10.0,15.5,-101.59252645502646,-10.222751322751323,19.678637566137567,0.004484304932735426,0.3333333333333333,2896.6030104244523,2896.6030104244523,2896.6030104244523,3154.17976659543,0.918337959396334,40.67767042165798,10.677670421657979,2,1,39.18754577636719,-108.74336242675781,"POLYGON ((-108.74405 39.18686, -108.74405 39.1..."


In [15]:
gis_df = pd.read_csv("./cell_gis.csv")
gis_df["geometry"] = gis_df["geometry"].apply(wkt.loads)

gis_df = gpd.GeoDataFrame(gis_df, geometry="geometry", crs="EPSG:4326")

In [16]:
gdf_m = grid_geo_data[["cell_name", "distance_to_cell", "geometry"]].copy()
proj_crs = grid_geo_data.estimate_utm_crs()
gdf_m = gdf_m.to_crs(proj_crs)

# Add cell origin so that convex-hull in worst case will start at the cell
gis_df_add = gis_df[gis_df.cell_name.isin(gdf_m.cell_name.to_list())][["cell_name", "geometry"]].copy()
#gis_df_add.rename(columns = {"Name" : "cell_name"}, inplace = True)
gis_df_add["distance_to_cell"] = 0.0
gis_df_add = gis_df_add.to_crs(proj_crs)

gdf_m["pct_rank"] = gdf_m.groupby("cell_name")["distance_to_cell"].rank(method="first", pct=True)

# keep the top 98% *closest* samples per cell (i.e., smallest distances)
gdf_m = gdf_m[gdf_m["pct_rank"] <= 0.98].drop(columns="pct_rank")

gdf_m = pd.concat([gdf_m, gis_df_add], ignore_index=True)
# 2) Make hulls per cell
hulls_m = (
    gdf_m.groupby('cell_name')['geometry']
         .apply(lambda s: s.unary_union.convex_hull)
         .reset_index()
)

hulls_m = gpd.GeoDataFrame(hulls_m, geometry='geometry', crs=proj_crs)

# 3) (Optional) compute area in km²
hulls_m['area_km2'] = hulls_m.area / 1e6

# 4) Reproject back to WGS84 for web maps
hulls = hulls_m.to_crs(4326)

del gdf_m

/var/folders/vx/v2s2cp2d233bk24wrz0wfw8h0000gn/T/ipykernel_56658/468044235.py:20: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  .apply(lambda s: s.unary_union.convex_hull)
/Users/stephendillon/anaconda3/envs/geoEnv/lib/python3.11/site-packages/shapely/set_operations.py:421: RuntimeWarning: invalid value encountered in unary_union
  return lib.unary_union(collections, **kwargs)


In [20]:
hulls.to_csv("cell_hulls.csv", index = False)

In [2]:
df = pd.read_csv("./cell_impacts.csv")

In [6]:
df.head(10)

,Unnamed: 0,traffic_data,traffic_voice,total_cell_traffic_voice_sa,total_cell_traffic_voice_nsa,total_cell_traffic_voice_lte,total_cell_traffic_voice_umts,total_cell_traffic_voice_gsm,total_sa_impact_voice_perc,total_nsa_impact_voice_perc,total_lte_impact_voice_perc,total_umts_impact_voice_perc,total_gsm_impact_voice_perc,total_cell_traffic_data_sa,total_cell_traffic_data_nsa,total_cell_traffic_data_lte,total_cell_traffic_data_umts,total_cell_traffic_data_gsm,total_sa_impact_data_perc,total_nsa_impact_data_perc,total_lte_impact_data_perc,total_umts_impact_data_perc,total_gsm_impact_data_perc,sa_on_cell,nsa_on_cell,lte_on_cell,umts_on_cell,gsm_on_cell,sa_on_impact_cell,nsa_on_impact_cell,lte_on_impact_cell,umts_on_impact_cell,gsm_on_impact_cell,market,vendor,cell_name,cell_pci,cell_lat,cell_lon,cell_elec_tilt,cell_mech_tilt,cell_band,cell_tech,site,cell,cell_impact_tech,cell_impact_name,cell_impact_pci,cell_impact_lat,cell_impact_lon,cell_impact_elec_tilt,cell_impact_mech_tilt,cell_impact_band,impact_cell,impact_site,co_site,co_sectored,distance,drops_voice,neighbor_relation,total_cell_traffic_voice,total_cell_traffic_data,relation_impact_voice_perc,relation_impact_data_perc
0,0,1,0,2779,0,0,0,0,0.0,0,0,0,0,297095,0,0,0,0,0.000003,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00141B_n70_AWS-4_UL15_2,199,39.743755,-104.965646,2.0,0,n70_AWS-4_UL5,SA,DNDEN00141B,DNDEN00141B_2,SA,DNDEN00051D_n71_F-G_3,698,39.731486,-104.930944,4.0,0,n71_F-G,DNDEN00051D_3,DNDEN00051D,N,N,3265.848572,0,NaN,2779,297095,0.0,0.000003
1,1,1,0,2966,0,0,0,0,0.0,0,0,0,0,353514,0,0,0,0,0.000003,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00124A_n66_G_1,855,39.775778,-104.992833,2.0,0,n66_G,SA,DNDEN00124A,DNDEN00124A_1,SA,DNDEN00163A_n71_F-G_3,605,39.732209,-104.941152,3.0,0,n71_F-G,DNDEN00163A_3,DNDEN00163A,N,N,6556.641729,0,NaN,2966,353514,0.0,0.000003
2,2,105,0,1694,0,0,0,0,0.0,0,0,0,0,125765,0,0,0,0,0.000835,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00357A_n71_G_1,375,38.880831,-104.821506,2.0,0,n71_G,SA,DNDEN00357A,DNDEN00357A_1,SA,DNDEN00357A_n70_AWS-4_UL15_3,377,38.880831,-104.821506,2.0,0,n70_AWS-4_UL5,DNDEN00357A_3,DNDEN00357A,Y,N,0.000000,0,NaN,1694,125765,0.0,0.000835
3,3,1,0,465,0,0,0,0,0.0,0,0,0,0,58416,0,0,0,0,0.000017,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00391C_n70_AWS-4_UL15_3,0,39.937461,-104.948093,2.0,0,n70_AWS-4_UL5,SA,DNDEN00391C,DNDEN00391C_3,SA,DNDEN00115A_n70_AWS-4_UL15_1,327,39.882205,-104.887197,2.0,0,n70_AWS-4_UL5,DNDEN00115A_1,DNDEN00115A,N,N,8045.399516,0,NaN,465,58416,0.0,0.000017
4,4,1,0,2423,0,0,0,0,0.0,0,0,0,0,262019,0,0,0,0,0.000004,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00017A_n71_F-G_2,106,39.675333,-104.873361,9.0,0,n71_F-G,SA,DNDEN00017A,DNDEN00017A_2,SA,DNDEN00059A_n70_AWS-4_UL15_3,191,39.635528,-104.790222,2.0,0,n70_AWS-4_UL5,DNDEN00059A_3,DNDEN00059A,N,N,8381.407584,0,NaN,2423,262019,0.0,0.000004
5,5,1,0,481,0,0,0,0,0.0,0,0,0,0,50694,0,0,0,0,0.000020,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00142A_n70_AWS-4_UL15_2,259,39.755744,-105.010511,2.0,0,n70_AWS-4_UL5,SA,DNDEN00142A,DNDEN00142A_2,SA,DNDEN00056A_n71_F-G_2,442,39.776917,-105.011911,2.0,0,n71_F-G,DNDEN00056A_2,DNDEN00056A,N,N,2357.369082,0,NaN,481,50694,0.0,0.000020
6,6,2,0,2241,0,0,0,0,0.0,0,0,0,0,184477,0,0,0,0,0.000011,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00059A_n70_AWS-4_UL15_1,189,39.635528,-104.790222,3.0,0,n70_AWS-4_UL5,SA,DNDEN00059A,DNDEN00059A_1,SA,DNDEN00087A_n70_AWS-4_UL15_3,512,39.723819,-104.792461,2.0,0,n70_AWS-4_UL5,DNDEN00087A_3,DNDEN00087A,N,N,9819.380937,0,NaN,2241,184477,0.0,0.000011
7,7,16,0,879,0,0,0,0,0.0,0,0,0,0,72563,0,0,0,0,0.000220,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,samsung,DNDEN00016C_n71_F-G_2,394,39.748864,-104.999761,4.0,0,n71_F-G,SA,DNDEN00016C,DNDEN00016C_2,SA,DNDEN00140B_n71_F-G_3,449,39.779528,-104.943222,5.0,0,n71_F-G,DNDEN00140B_3,DNDEN00140B,N,N,5914.382291,0,NaN,879,72563,0.0,0.000220
8,8,3,0,437,0,0,0,0,0.0,0,0,0,0,61637,0,0,0,0,0.000049,0,0,0,0,Y,N,N,N,N,Y,N,N,N,N,denver,sam

In [5]:
df['total_sa_impact_voice_perc'] = df['traffic_voice'] / df['total_cell_traffic_voice']
df['total_sa_impact_data_perc'] = df['traffic_data'] / df['total_cell_traffic_data']
df['relation_impact_voice_perc'] = df['traffic_voice'] / df['total_cell_traffic_voice']
df['relation_impact_data_perc'] = df['traffic_data'] / df['total_cell_traffic_data']

In [9]:
df = df[['traffic_data', 'traffic_voice',
       'total_cell_traffic_voice_sa', 'total_cell_traffic_voice_nsa',
       'total_cell_traffic_voice_lte', 'total_cell_traffic_voice_umts',
       'total_cell_traffic_voice_gsm', 'total_sa_impact_voice_perc',
       'total_nsa_impact_voice_perc', 'total_lte_impact_voice_perc',
       'total_umts_impact_voice_perc', 'total_gsm_impact_voice_perc',
       'total_cell_traffic_data_sa', 'total_cell_traffic_data_nsa',
       'total_cell_traffic_data_lte', 'total_cell_traffic_data_umts',
       'total_cell_traffic_data_gsm', 'total_sa_impact_data_perc',
       'total_nsa_impact_data_perc', 'total_lte_impact_data_perc',
       'total_umts_impact_data_perc', 'total_gsm_impact_data_perc',
       'sa_on_cell', 'nsa_on_cell', 'lte_on_cell', 'umts_on_cell',
       'gsm_on_cell', 'sa_on_impact_cell', 'nsa_on_impact_cell',
       'lte_on_impact_cell', 'umts_on_impact_cell', 'gsm_on_impact_cell',
       'market', 'vendor', 'cell_name', 'cell_pci', 'cell_lat', 'cell_lon',
       'cell_elec_tilt', 'cell_mech_tilt', 'cell_band', 'cell_tech', 'site',
       'cell', 'cell_impact_tech', 'cell_impact_name', 'cell_impact_pci',
       'cell_impact_lat', 'cell_impact_lon', 'cell_impact_elec_tilt',
       'cell_impact_mech_tilt', 'cell_impact_band', 'impact_cell',
       'impact_site', 'co_site', 'co_sectored', 'distance', 'drops_voice',
       'neighbor_relation', 'total_cell_traffic_voice',
       'total_cell_traffic_data', 'relation_impact_voice_perc',
       'relation_impact_data_perc']]

In [10]:
df.to_csv("./cell_impacts.csv")